In [23]:
import numpy as np
import scipy as sp
import pandas as pd
import numpy_financial as nf


In [77]:
def computeFinancial(purchasePrice,capRate,intestRate,ltv,expenseRate=35.0):
    """
    compute financial
    purchase price - [$]
    capRate    - [%]
    intestRate - [%]
    ltv        - [%]
    """
    pmt = -1 * nf.pmt(intestRate/12./100.,30*12,purchasePrice * ltv/100.)
    out=dict()
    out['NOI'] = (purchasePrice * capRate/100.) 
    out["Rent Roll"]= out['NOI'] /(1-expenseRate/100)
    out['DebtSrv']  = (pmt * 12)
    out['dscr']     = out['NOI']/ out['DebtSrv']
    return out

In [96]:
import ipywidgets as widgets
from IPython.display import clear_output

import pprint
pp = pprint.PrettyPrinter(indent=2)

currentData  = {}
proformaData ={}
wNumUnits = widgets.BoundedIntText(value=4,min=0,max=16,step=1,description='NumUnits:')
wPrice = widgets.BoundedIntText(
    value=1e6,
    min=0,
    max=5e6,
    step=1e5,
    description='Price:',
    disabled=False,
    readout_format="$%.2d"
)
wLtv = widgets.BoundedIntText(
    value=75,
    min=0,
    max=100,
    step=5,
    description='LTV:',
    disabled=False,
    readout_format="$.2d%"
)
wCap = widgets.BoundedFloatText(
    value=4.,
    min=2,
    max=10,
    step=0.1,
    description='Cap:',
    disabled=False,
    readout_format="$.2d%"
)

wExpRate = widgets.BoundedFloatText(
    value=35.,
    min=30,
    max=50,
    step=1,
    description='Exp[%]:',
    disabled=False,
    readout_format="$.2d%"
)

wInterestRate= widgets.BoundedFloatText(
    value=3.5,
    min=2,
    max=10,
    step=0.1,
    description='interest[%]:',
    disabled=False,
    readout_format="$.2d%"
)
wCapPrj = widgets.BoundedFloatText(
    value=4.,
    min=2,
    max=10,
    step=0.1,
    description='Cap:',
    disabled=False,
    readout_format="$.2d%"
)

wExpRatePrj = widgets.BoundedFloatText(
    value=35.,
    min=30,
    max=50,
    step=1,
    description='Exp[%]:',
    disabled=False,
    readout_format="$.2d%"
)
wOut    = widgets.Output(layout={'border': '1px solid black'})
wOutPrj = widgets.Output(layout={'border': '1px solid black'})
wOutDelt= widgets.Output(layout={'border': '1px solid black'})


def handlerCompute(change):
    if change["name"] == "value":        
        currentData = computeFinancial(
            capRate       = wCap.value,
            purchasePrice = wPrice.value, 
            intestRate    = wInterestRate.value,
            ltv           = wLtv.value,
            expenseRate   = wExpRate.value)
        with wOut:            
            clear_output()
            display.display(currentData)
            
        proformaData = computeFinancial(
            capRate       = wCapPrj.value,
            purchasePrice = wPrice.value, 
            intestRate    = wInterestRate.value,
            ltv           = wLtv.value,
            expenseRate   = wExpRatePrj.value)  
        with wOutPrj:                        
            clear_output()
            display.display(proformaData)
        with wOutDelt:                   
            clear_output()
            data = dict()
            
            data["Exp Reduction"]   = (currentData["Rent Roll"] - currentData["NOI"]) - (proformaData["Rent Roll"] - proformaData["NOI"])
            data["Rent Roll Delta"] = (proformaData["Rent Roll"]-currentData["Rent Roll"])
            display.display("Rent roll increase per month per unit:%.2f [%.1f]%%"%(data["Rent Roll Delta"]/12.0/wNumUnits.value, 100*data["Rent Roll Delta"]/currentData["Rent Roll"]))
            display.display("Expense reduction per year(%.0f)/month(%.0f)"%(data["Exp Reduction"],data["Exp Reduction"]/12.))
            
wCap.observe(handlerCompute)
wPrice.observe(handlerCompute)
wLtv.observe(handlerCompute)
wInterestRate.observe(handlerCompute)
wExpRate.observe(handlerCompute)

wCapPrj.observe(handlerCompute)
wExpRatePrj.observe(handlerCompute)
wNumUnits.observe(handlerCompute)
widgets.VBox([
    wNumUnits,
    wPrice,
    widgets.HBox([wCap,wCapPrj]),
    widgets.HBox([wExpRate,wExpRatePrj]),
    wInterestRate,
    wLtv,    
    widgets.HBox([wOut,wOutPrj]),
    wOutDelt,
    ])

